In [ ]:
import numpy as np
import pandas as pd
import pickle
from sklearn import datasets # sklearn comes with some toy datasets to practice
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
import spotipy
import pandas as pd
import json
from spotipy.oauth2 import SpotifyClientCredentials
import config
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))
from IPython.display import IFrame
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 

# Import Model & Scaler

In [4]:
def load(filename = 'kmeans_4.pickle'):
    try:
        with open(filename, 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        print('File not found!')

In [5]:
load(filename = 'kmeans_4.pickle')

KMeans(n_clusters=19, n_init='auto', random_state=1234)

In [6]:
model = load(filename = 'spotify_model.pickle')

In [7]:
def load(filename = 'scaler.pickle'):
    try:
        with open(filename, 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        print('File not found!')

In [8]:
load(filename = 'scaler.pickle')

StandardScaler()

In [9]:
scaler = load(filename = 'scaler.pickle')

# Import DataFrames

In [10]:
import pandas as pd


hot100 = pd.read_csv("hot100.csv")
country = pd.read_csv("country_df.csv")
X = pd.read_csv('X.csv')

In [11]:
X

,danceability,energy,loudness,tempo,valence,acousticness,cluster,id
0,0.796,0.865,-6.760,130.012,0.768,0.17900,4,5qQLBOEAelSN7JWqX3RIIv
1,0.776,0.714,-5.705,161.015,0.433,0.02830,2,4rSmXn2TIKt6iBeBDM4uOs
2,0.960,0.670,-7.035,133.966,0.800,0.05980,11,6R5iuYqG0HsnC41RT3lzqX
3,0.752,0.571,-6.422,169.155,0.492,0.07230,2,6FGPpwHlUBnl0TLGKyN4Nl
4,0.774,0.700,-7.334,81.990,0.624,0.16300,19,3ilgxZl5g7MOQCUiky3VOG
...,...,...,...,...,...,...,...,...
907,0.720,0.932,-4.204,130.928,0.918,0.09760,4,4cSSAsYt4afPzyaBdO0WVs
908,0.667,0.821,-5.565,121.027,0.638,0.09410,14,3fGkWWpjU3xghZbKa2rQUW
909,0.704,0.934,-4.528,121.014,0.494,0.02850,14,1JlczCvmw3K2MizmtDvfp3
910,0.716,0.695,-5.250,117.966,0.684,0.00643,14,0oUlljswfmHwdNpjHnn8Ii


# Prepare the Model

In [45]:
title = input("Please enter a title: ")

Please enter a title: cruel summer


In [46]:

results = sp.search(q=f"{title}", limit=1, market="GB")
audio_features = sp.audio_features(results["tracks"]["items"][0]["id"])[0]
new_features = {'danceability': audio_features['danceability'], 'energy': audio_features['energy'], 'loudness': audio_features['loudness'], 'tempo': audio_features['tempo'], 'valence': audio_features['valence'], 'acousticness': audio_features['acousticness']}
new_features_df = pd.DataFrame([new_features])
scaler.transform(new_features_df)


array([[-0.25292575,  0.59396359,  0.64329226,  1.66033522,  0.82045942,
        -0.88570279]])

In [47]:
X[X['cluster']==2]

,danceability,energy,loudness,tempo,valence,acousticness,cluster,id
1,0.776,0.714,-5.705,161.015,0.433,0.0283,2,4rSmXn2TIKt6iBeBDM4uOs
3,0.752,0.571,-6.422,169.155,0.492,0.0723,2,6FGPpwHlUBnl0TLGKyN4Nl
7,0.819,0.538,-6.811,125.026,0.542,0.1410,2,5jGcOhXpPE07vrSe23198B
15,0.823,0.567,-9.571,136.006,0.545,0.1230,2,001TLpmtuQMWJYyE7q9ayM
21,0.828,0.785,-5.719,145.044,0.575,0.2750,2,2AQjQLPLY5EZ667L2Z9M0D
...,...,...,...,...,...,...,...,...
836,0.899,0.348,-16.467,140.044,0.398,0.1880,2,33YgGfIfF8ntJGWD35WkCu
841,0.782,0.698,-10.755,150.043,0.502,0.0249,2,3sMNkmHA0fZkt2F5pqtVGe
846,0.921,0.522,-6.969,139.940,0.332,0.0611,2,7h89KNdndkucY4nDlwr31b
851,0.701,0.536,-12.523,160.199,0.382,0.1130,2,7wc0bSJHs36VJ0po9UCvHj


In [49]:
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id, #first part does not change
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )



In [53]:
def recommend_song(input_song):
    input_song = input_song.lower()
    while True:
        match, score = process.extractOne(input_song, hot100['title'].str.lower().values)
        threshold = 80

        if score >= threshold:
            # Get a recommendation 
            recommendation = hot100[hot100['title'].str.lower() != match].sample(1)['title'].values[0]

            print(f"If you liked '{match.capitalize()}', you might also like '{recommendation.capitalize()}'!")
            
            
            another_song = input(f"Do you want to add another song? YES/NO ")
            if another_song == "NO":
                print(f"Ok, you lamo lol ")
                break
            
            elif another_song == "YES":
                input_song = input("Enter another song title: ")
            else:
                print("Invalid input. Please enter YES or NO.")
            
        else:
            results = sp.search(q=f"{input_song}", limit=1, market="GB")
            audio_features = sp.audio_features(results["tracks"]["items"][0]["id"])[0]
            new_features = {'danceability': audio_features['danceability'], 'energy': audio_features['energy'], 'loudness': audio_features['loudness'], 'tempo': audio_features['tempo'], 'valence': audio_features['valence'], 'acousticness': audio_features['acousticness']}
            new_features_df = pd.DataFrame([new_features])
            transformed_features = scaler.transform(new_features_df)
            prediction = model.predict(transformed_features)[0]
            filtered_songs = X[X['cluster'] == prediction]
            new_recommend = filtered_songs.sample()
            track_id = new_recommend['id'].values[0]
            

            
            if len(filtered_songs) > 0:
                # If there are songs, get a random sample
                recommended_song = filtered_songs.sample(1)

                # Print the recommended song
                print(f"If you liked '{input_song.capitalize()}', you might also like '!")
                display(play_song(track_id))

                          
            else:
                # If no songs meet the condition, print a message
                print("Sorry, no recommendations found based on your input.")
                
            break
    
# Let user input a song
user_input = input("Enter a song title: ")


#Calling the function
recommend_song(user_input)

Enter a song title: sandstorm
If you liked 'Sandstorm', you might also like '!


/Users/dominik/anaconda3/lib/python3.10/site-packages/sklearn/base.py:420: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
